In [13]:
import os
import sys

project_path = "/home/moonchild/PythonProjects/Preconditioning/opencampus-preconditioner-ai-project"

# Append the project directory to sys.path
if project_path not in sys.path:
    sys.path.append(project_path)


In [18]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [17]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Activation, BatchNormalization, ZeroPadding2D, Input
from tensorflow.keras.regularizers import l2

import importlib

import matrixlib.io
import matrixlib.util
import matrixlib.plot
import matrixlib.generate

import CNN.models
import CNN.train
import CNN.evaluation
import matrixlib.generate

# reload the library when changes were made to it
importlib.reload(matrixlib.io)
importlib.reload(matrixlib.util)
importlib.reload(matrixlib.plot)
importlib.reload(matrixlib.generate)

importlib.reload(CNN.models)
importlib.reload(CNN.train)
importlib.reload(CNN.evaluation)
importlib.reload(matrixlib.generate)

ImportError: cannot import name 'Input' from 'keras.models' (/home/moonchild/PythonProjects/Preconditioning/opencampus-preconditioner-ai-project/matrix_venv/lib/python3.12/site-packages/keras/api/models/__init__.py)